In [9]:
# pip install pandas

# pip install scikit-learn

!pip install -r requirements_short.txt


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/7d/af/03d3a7d5719d00486c296ddd876e6f07a681bc4e079cb45348d2f261a748/scikit_learn-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.3 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/b5/24/aaccf324ce862bb82277e8814d2aebbb2a2c160d04e95aa2b8c9dc3137a9/matplotlib-3.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/18/44/7e8d208eb59a8224fcc474415104f13be9b378be8da63f76dfde12ec2b44/scipy-1.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata


In [4]:
import pandas as pd
#import pyreadr #package to allow us to read in .rds data files (native R datafile)
from nltk.stem.snowball import SnowballStemmer
import re
from tqdm import tqdm
from collections import OrderedDict
from collections import defaultdict
from collections import namedtuple
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import WhitespaceTokenizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import random
from math import lgamma
from sklearn import linear_model
import matplotlib.pyplot as plt
import datetime
import pickle

In [5]:
# To read a .rds file

with open('french_election_broad(1).pkl', 'rb') as file:
    # Load the object from the pickle file
    df = pickle.load(file)

print(df.head())

print(len(df))

,source,target,weight,text,month
0,Mehetia1,None,1,@demontvalon1 @EmmanuelMacron Etudiants : La #...,December
1,LORZMichael_Off,None,1,Une honte. Hidalgo devrait être viré au plus v...,December
2,Thomas_France75,None,1,Ne plus fermer les yeux #presidentielles2022🇫🇷👇🏼,December
3,Fmercier2017,None,1,"Le nucléaire est certes productif, mais trop p...",December
4,BerbreReineKah1,None,1,Que c'est beau de rêver😢\nElle est belle la st...,December


1263026

In [9]:
df = df[~df['text'].str.startswith(('rt', 'RT'))]

In [10]:
len(df)

1262531

In [11]:
df.reset_index(inplace = True, drop = True)

In [12]:
len(df)

1262531

In [13]:
# df = df.loc[df.text.drop_duplicates().index] # removing duplicate observations (tweets)

In [ ]:
# df.to_pickle('french_election_broad_no_duplicates.pkl')

In [14]:
df['text_original'] = df['text']
df['text'] = df['text'].str.lower()

/tmp/ipykernel_2130/4161460516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_original'] = df['text']
/tmp/ipykernel_2130/4161460516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()


In [15]:
# Remove mentions (posts that start with a "@some_user_name ")
df['text'] = df['text'].str.replace("\\B@\\w+|^@\\w+", "", regex = True)
# Change ampersands to "and"
df['text'] = df['text'].str.replace("&amp;", "and")
# Remove the "RT" and "via" (old retweet style)
df['text'] = df['text'].str.replace("(^RT|^via)((?:\\b\\W*@\\w+)+)","", regex=True, case=False)
# Remove URLs             
df['text'] = df['text'].str.replace("(https|http)?:\\/\\/(\\w|\\.|\\/|\\?|\\=|\\&|\\%)*\\b", "", regex = True)
# Keep ASCII only (removes Cyrillic, Japanese characters, etc.)
df['text'] = df['text'].str.replace("[^ -~]", "", regex = True)
# Remove double+ spaces (e.g. "build   the wall" to "build the wall")
df['text'] = df['text'].str.replace("\\s+", " ", regex = True)

/tmp/ipykernel_2130/854266914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace("\\B@\\w+|^@\\w+", "", regex = True)
/tmp/ipykernel_2130/854266914.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace("&amp;", "and")
/tmp/ipykernel_2130/854266914.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [16]:
# The keywords that add through the keyword discovery process

reference_words_seed  = ["presidentielle","presidentielles"] #first word we started with

#reference_words_expanded = ["covid-19", "coronavirus"] #expanded list of keywords (just to show how it works)
reference_words_expanded = []

In [17]:
# The keywords that add through the keyword discovery process
reference_words_excluded = [] #initial
reference_words_excluded_expanded = [] #expanded


In [18]:
# Put the seed words and keywords added through discovery together

reference_words = reference_words_seed + reference_words_expanded
reference_words = "|".join(reference_words)

reference_words_excluded = reference_words_excluded + reference_words_excluded_expanded
reference_words_excluded = "|".join(reference_words_excluded)


In [19]:
reference_words

'presidentielle|presidentielles'

In [20]:
reference_words_excluded

''

In [21]:
# Will there be any keywords we want to exclude? this is simply a test 
excluded_words = len(reference_words_excluded.replace("|", "")) > 0 #checks if there are more chars than 0, returns a bool

print(excluded_words)

False


In [22]:
# Find tweets that contain any keywords that we want to include

df['includes'] = df['text'].str.contains(pat = reference_words, regex = True, case = False)


# Find tweets that contain any keywords that we want to exclude
df['excludes'] = False
if excluded_words:
    df['excludes']  = df.loc[df['text'].str.contains(pat = reference_words_excluded, regex = True, case = False),:]


/tmp/ipykernel_2130/609187822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['includes'] = df['text'].str.contains(pat = reference_words, regex = True, case = False)
/tmp/ipykernel_2130/609187822.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['excludes'] = False


In [23]:
# Just a bunch of TRUE and FALSE for the posts that include and exclude keywords
df['reference_set'] = (df['includes']== True) & (df['excludes'] == False) #all rows with the included and without the excluded
df['reference_set'] = df['reference_set'] *1 #make into numeric


/tmp/ipykernel_2130/1320542145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reference_set'] = (df['includes']== True) & (df['excludes'] == False) #all rows with the included and without the excluded
/tmp/ipykernel_2130/1320542145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reference_set'] = df['reference_set'] *1 #make into numeric


In [24]:
#how many of each?
df['reference_set'].value_counts()


0    976332
1    286199
Name: reference_set, dtype: int64

In [25]:
# We'll remove the keywords from the tweets so that the machine learning model
# needs to use the words in each tweet that aren't those keywords to predict
# whether it belongs in the target set or not the target set

remove_keywords_from_tweets = True

#make a reg-ex list of all the inclusion and exclusion words

if excluded_words: #if there are exclusion words
    all_current_keywords = reference_words + "|" + reference_words_excluded
else:
    all_current_keywords = reference_words


df['text_preprossed'] = df['text']

if remove_keywords_from_tweets:
    df['text_preprossed'] = df['text'].str.replace(all_current_keywords, "", regex = True, case = False)


/tmp/ipykernel_2130/4067524263.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_preprossed'] = df['text']
/tmp/ipykernel_2130/4067524263.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_preprossed'] = df['text'].str.replace(all_current_keywords, "", regex = True, case = False)


In [26]:
tokenizer = WhitespaceTokenizer()
ps = PorterStemmer()

In [27]:
pre_prossed_sents =[]
for sent in tqdm(df['text_preprossed']):
    words = tokenizer.tokenize(sent)
    words = [re.sub(r'\d+', '', word) for word in words] #removing tokens that are only words 
    words = [x for x in words if x] #removing empty strings
    sent_stem = [ps.stem(word) for word in words]
    
    sent_done = " ".join(sent_stem)
    pre_prossed_sents.append(sent_done)


  0%|          | 327/1262531 [00:00<06:26, 3264.89it/s]

100%|██████████| 1262531/1262531 [06:17<00:00, 3348.74it/s]


In [28]:
df['text_pre_stem'] = pre_prossed_sents

/tmp/ipykernel_2130/1358738971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_pre_stem'] = pre_prossed_sents


In [30]:
len(df)

1262531

In [31]:
df2 = df[~df['text'].str.startswith(('rt', 'RT'))]

In [32]:
len(df2)

1262441

In [29]:
# Consider saving a csv at this time

df.to_pickle('election_processed1.pkl')

In [33]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words_french = list(set(stopwords.words('french')))

[nltk_data] Downloading package stopwords to /home/ucloud/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [55]:

#  make a copy of DF for training only -- very small, only 10k 

df_train = df.copy()

sample_size = 5000
df_train = df.sample(n=sample_size, random_state=42)
df_train.reset_index(inplace = True, drop = True)

vectorizer = CountVectorizer(min_df = 10,
                           max_df = 0.999,
                           stop_words=stop_words_french,
                          ngram_range = (1,2)) #set for larger n-grams

corpus = vectorizer.fit_transform(df_train['text_pre_stem'])

DTM_dict = {"DTM":corpus,
               "labels":list(df_train['reference_set'])} 

# Determine how many tweets to sample for the training data

# n_search = 3000
n_search = min(sum(df_train['reference_set']==1)* 2, 50000)

#NB: Greg uses *20 - but that would take too long to run for the purposes of this exercise

# Create lists of indices, indicating which belong to the reference set (those labelled to be about covid)
#   and which do not belong to the reference set

reference_ids = list(df_train[df_train['reference_set']==1].index)
search_ids = list(df_train[df_train['reference_set']==0].index)

# Taking a random sample of the search indices, i.e. those that do not belong to the reference set
search_ids_sample = random.choices(search_ids, k=n_search)
ids = reference_ids + search_ids_sample # putting them together to create the full list of training set indices

# Checking amounts
print(len(reference_ids),len(search_ids_sample),len(ids))

# Saving the training data in a dict object
DTM_train = {"DTM" : corpus[ids,:],
            "labels": list(df_train['reference_set'][ids])}
        
# Defining the classifier

#linear lasso
#clf = linear_model.LassoCV(n_jobs=-1, verbose=1) #this was a mistake - should be log reg - small mistake though

#logistic reg with lasso penalty - this will actually give probs! e
clf = linear_model.LogisticRegressionCV(penalty="l1", n_jobs = -1, solver = "saga", max_iter=10000, verbose = 1)

# Fitting the model. This takes some time. 

clf.fit(DTM_train['DTM'], DTM_train['labels']) 


1112 2224 3336
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
Epoch 1, change: 1.00000000
Epoch 2, change: inf
Epoch 1, change: 1.00000000
convergence after 3 epochs took 0 seconds
Epoch 1, change: 1.00000000
Epoch 1, change: 1.00000000
Epoch 1, change: 1.00000000
Epoch 2, change: inf
convergence after 1 epochs took 0 seconds
Epoch 2, change: inf
Epoch 2, change: inf
Epoch 2, change: inf
convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
Epoch 1, change: 1.00000000
convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
Epoch 2, change: 0.15841317
Epoch 1, change: 1.00000000
Epoch 3, change: 0.06804118
Epoch 2, change: 0.19699336
convergence after 1 epochs took 0 seconds
Epoch 4, cha

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


Epoch 26, change: 0.00758184
Epoch 23, change: 0.00906528
Epoch 13, change: 0.02581424
Epoch 33, change: 0.00451157
Epoch 16, change: 0.02076663
Epoch 27, change: 0.00684647
Epoch 24, change: 0.00887033
Epoch 34, change: 0.00427755
Epoch 14, change: 0.02317889
Epoch 28, change: 0.00681642
Epoch 35, change: 0.00396928
Epoch 17, change: 0.01811412
Epoch 25, change: 0.00817556
Epoch 15, change: 0.02051681
Epoch 36, change: 0.00381557
Epoch 29, change: 0.00597692
Epoch 26, change: 0.00763085
Epoch 18, change: 0.01775087
Epoch 16, change: 0.01780105
Epoch 37, change: 0.00362192
Epoch 30, change: 0.00567087
Epoch 27, change: 0.00731130
Epoch 19, change: 0.01579001
Epoch 17, change: 0.01564036
Epoch 38, change: 0.00338912
Epoch 31, change: 0.00536545
Epoch 28, change: 0.00703543
Epoch 20, change: 0.01510032
Epoch 39, change: 0.00307412
Epoch 18, change: 0.01457442
Epoch 32, change: 0.00491998
Epoch 29, change: 0.00655788
Epoch 21, change: 0.01389709
Epoch 40, change: 0.00299596
Epoch 33, chan

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.1min


Epoch 138, change: 0.00010344
Epoch 1211, change: 0.00011313
Epoch 1715, change: 0.00012052
Epoch 1212, change: 0.00011302
Epoch 139, change: 0.00010340
Epoch 1716, change: 0.00012051
Epoch 1213, change: 0.00011305
Epoch 1717, change: 0.00012044
Epoch 140, change: 0.00010337
Epoch 1214, change: 0.00011295
Epoch 1718, change: 0.00012041
Epoch 141, change: 0.00010337
Epoch 1215, change: 0.00011297
Epoch 1719, change: 0.00012038
Epoch 142, change: 0.00010336
Epoch 1216, change: 0.00011296
Epoch 1720, change: 0.00012034
Epoch 143, change: 0.00010328
Epoch 1217, change: 0.00011295
Epoch 1721, change: 0.00012030
Epoch 144, change: 0.00010329
Epoch 1218, change: 0.00011274
Epoch 1722, change: 0.00012029
Epoch 145, change: 0.00010328
Epoch 1219, change: 0.00011275
Epoch 1723, change: 0.00012022
Epoch 146, change: 0.00010318
Epoch 1220, change: 0.00011275
Epoch 1724, change: 0.00012021
Epoch 147, change: 0.00010317
Epoch 1221, change: 0.00011266
Epoch 1725, change: 0.00012017
Epoch 148, change:

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min finished


LogisticRegressionCV(max_iter=10000, n_jobs=-1, penalty='l1', solver='saga',
                     verbose=1)

In [97]:
# Testing performance now:

# Making predictions on the training data
train_predictions = clf.predict(DTM_train['DTM'])

# Calculating evaluation metrics for the training set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_train = accuracy_score(DTM_train['labels'], train_predictions)
precision_train = precision_score(DTM_train['labels'], train_predictions)
recall_train = recall_score(DTM_train['labels'], train_predictions)
f1_train = f1_score(DTM_train['labels'], train_predictions)

print("Training Accuracy:", accuracy_train)
print("Training Precision:", precision_train)
print("Training Recall:", recall_train)
print("Training F1 Score:", f1_train)

Training Accuracy: 0.934652278177458
Training Precision: 0.9496981891348089
Training Recall: 0.8489208633093526
Training F1 Score: 0.8964862298195632


In [99]:
#  now, scaling up to the whole df 
corpus2 = vectorizer.transform(df['text_pre_stem'])

# Making predictions on the full dataset
predictions = clf.predict(corpus2)

# Check if all predictions are the same (we don't want this to be zero)
# If so, there is a problem: potentially too few observations in the reference
# set relative to the search set

np.std(predictions)

# Set a threshold

threshold = 0.25



In [73]:
# Saving variables so far in a dict

DTM_full = {'DTM':corpus2,
            'raw_preds': predictions,
            'preds': (predictions > threshold) * 1 ,
            'reference': list(df['reference_set']==1),
            'reference_ids': reference_ids}

In [74]:
# Predict whether each tweet is in the reference set based on a predicted
# probability, where Pr(reference_set = 1) > threshold. Keep this low if want
# more tweets to get into the target set

In [75]:
set_var = []

for i in range(0,len(DTM_full['preds'])):
    if i in reference_ids: #if the obs is in the reference group
        set_var.append('Reference')
    
    elif DTM_full['preds'][i] == 1:
        set_var.append('Target')
    
    elif DTM_full['preds'][i] == 0:
        set_var.append('Not target')
    
    else:
        set_var.append(None)
        

In [76]:
# Creating a new dataframe of predictions 

pred_df = pd.DataFrame([pd.Series(DTM_full['preds']),pd.Series(set_var)]).T

In [77]:
# Checking performance (only classification of the reference observations are relevant to check)

pd.crosstab(pred_df[0],pred_df[1])


1,Not target,Reference,Target
0,,,
0,897018,882,0
1,0,230,364401


In [78]:
# Saving the Reference/Target/Not target in the dict 

DTM_full['set_var'] = set_var

In [79]:
# Creating 3 sets of indices

target_ids = list(pd.Series(DTM_full['set_var']) == 'Target')
not_target_ids = list(pd.Series(DTM_full['set_var']) == 'Not target')
ref_ids = list(pd.Series(DTM_full['set_var']) == 'Reference')

In [80]:
# Creating statistics for the target, not_target and reference sets 

target_freq = np.sum(DTM_full['DTM'][target_ids,:],0) #how many times is each token used in the target documents
target_num_docs = np.sum(DTM_full['DTM'][target_ids,:] > 0, axis = 0) #how many target documents does each token appear in
target_num_docs_prop =  target_num_docs / sum(target_ids) #proportion of target docs with each token


not_target_freq = np.sum(DTM_full['DTM'][not_target_ids,:],0) #how many times is each token used in the not_target documents
not_target_num_docs = np.sum(DTM_full['DTM'][not_target_ids,:]> 0,axis = 0) #how many not_target documents does each token appear in
not_target_num_docs_prop =  not_target_num_docs / sum(not_target_ids) #proportion of not_target docs with each token


ref_freq = np.sum(DTM_full['DTM'][ref_ids,:],0) #how many times is each token used in the reference documents
ref_num_docs = np.sum(DTM_full['DTM'][ref_ids,:] > 0, axis = 0) #how many reference documents does each token appear in
ref_num_docs_prop =  ref_num_docs / sum(ref_ids) #proportion of reference docs with each token


# Saving the above in a dict 

d = {'target_freq' :target_freq.tolist()[0],
    'target_num_docs': target_num_docs.tolist()[0],
    'target_num_docs_prop': target_num_docs_prop.tolist()[0],
    'not_target_freq':not_target_freq.tolist()[0],
    'not_target_num_docs': not_target_num_docs.tolist()[0],
    'not_target_num_docs_prop': not_target_num_docs_prop.tolist()[0],
    'ref_freq':ref_freq.tolist()[0],
    'ref_num_docs': ref_num_docs.tolist()[0],
    'ref_num_docs_prop': ref_num_docs_prop.tolist()[0]}


In [81]:
# Creating a dataframe based on the dict and token names

df_prop = pd.DataFrame(d, index=vectorizer.get_feature_names_out())

In [82]:
# Subsetting observations that are target or not target at least once (probably all)
df_prop = df_prop.loc[(df_prop['target_freq']> 0) | (df_prop['not_target_freq'] > 0), :]

In [83]:
# Creating a column that is True if the token has a higher or equal proportion in the target set than in the not_target set. 
df_prop['target_larger'] = df_prop['target_num_docs_prop'] >= df_prop['not_target_num_docs_prop']


In [84]:
# Taking a look at our new dataframe
df_prop

,target_freq,target_num_docs,target_num_docs_prop,not_target_freq,not_target_num_docs,not_target_num_docs_prop,ref_freq,ref_num_docs,ref_num_docs_prop,target_larger
abord,489,486,0.001334,1280,1257,0.001401,2,2,0.001799,False
absolu,936,933,0.002560,3401,3371,0.003758,7,7,0.006295,False
abstent,3334,3198,0.008776,1642,1637,0.001825,0,0,0.000000,True
accord,3451,3318,0.009105,2456,2440,0.002720,6,6,0.005396,True
accueil,764,727,0.001995,2448,2439,0.002719,2,2,0.001799,False
...,...,...,...,...,...,...,...,...,...,...
zemmourvillepint zemmourpresid,141,141,0.000387,2365,2365,0.002637,8,8,0.007194,False
zemmourvspecress,378,378,0.001037,3452,3451,0.003847,0,0,0.000000,False
zozz,344,341,0.000936,4291,4256,0.004745,2,2,0.001799,False
zzz,494,489,0.001342,9876,9842,0.010972,7,7,0.006295,False


In [85]:
# Likelihood function

def llik(target_num_docs, nottarget_num_docs, target_num_docs_total, nottarget_num_docs_total):
    '''No docstring - you neew to see what it does :) '''
    x1 = ((lgamma(target_num_docs + 1) + lgamma(nottarget_num_docs + 1)) -
           lgamma(target_num_docs + nottarget_num_docs + 1 + 1))
    x2 = ((lgamma(target_num_docs_total - target_num_docs + 1) +
           lgamma(nottarget_num_docs_total - nottarget_num_docs + 1)) -
           lgamma(target_num_docs_total - target_num_docs +
          nottarget_num_docs_total - nottarget_num_docs + 1 + 1))
    llik = x1 + x2
    return llik

In [86]:
# Finding the total number of documents in the target and not_target sets 

target_num_docs_total = sum(target_ids)
nottarget_num_docs_total =  sum(not_target_ids) 

In [87]:
# Calculating the likelihood for each token

llik_list= []
for i in range(len(df_prop)):
    t =df_prop['target_num_docs'][i]
    nt =df_prop['not_target_num_docs'][i]
    l = llik(t,nt,target_num_docs_total,nottarget_num_docs_total)
    llik_list.append(l)


In [88]:
# Saving the likelihood scores 

df_prop['llik'] = llik_list


In [89]:
df_prop.to_pickle('output1.pkl')

In [ ]:
# # # # # # #  GOOD WORDS

In [210]:
df_prop.loc[df_prop['target_larger'] == True].sort_values('llik',ascending = False)[50:75]

,target_freq,target_num_docs,target_num_docs_prop,not_target_freq,not_target_num_docs,not_target_num_docs_prop,ref_freq,ref_num_docs,ref_num_docs_prop,target_larger,llik
macron macron,7666,7424,0.020373,5909,5645,0.006293,10,10,0.008993,True,-756078.348442
vote,25236,22598,0.062014,34597,31381,0.034984,30,25,0.022482,True,-756150.177194
retrait,8173,7100,0.019484,5834,5388,0.006007,1,1,0.000899,True,-756171.475699
rsultat,6635,6399,0.017560,4609,4524,0.005043,8,8,0.007194,True,-756210.944979
dmocrati,5414,5227,0.014344,3084,3054,0.003405,4,4,0.003597,True,-756214.134051
prsident rpubliqu,3352,3339,0.009163,1073,1071,0.001194,0,0,0.000000,True,-756234.949911
profit,2902,2842,0.007799,656,655,0.000730,1,1,0.000899,True,-756237.538374
question,7197,6910,0.018963,5406,5305,0.005914,7,7,0.006295,True,-756259.928909
er,9005,8673,0.023801,8008,7864,0.008767,13,12,0.010791,True,-756272.799596
dernier,4999,4902,0.013452,2870,2847,0.003174,5,5,0.004496,True,-756334.017423


In [225]:
list(see_in_context("emetour").iloc[20:40])

['#Zemmour2emeTour\n#JeNeSuisPasDextremeDroite #PourLesFrançaisOubliés #JeVoteZemmour',
 "@CharlotteGounot @EmmanuelMacron @BrunoLeMaire La vie est si belle qu'on risque la mort à chaque coin de trottoir ! #JeVoteZemmour #Zemmour #Zemmour2emeTour #MacronDegage",
 "@BFMTV Tout à fait d'accord 👍🏻\n\nZEMMOUR PRÉSIDENT ❤️\n\n#Zemmour2emeTour \n#JeVoteZemmour",
 'Je vote samedi en Guyane : je vote Zemmour #votezemmour #Zemmour #ZemmourPresident2022 #ZemmourProgramme #Zemmour2emeTour #Zemmour2022',
 '@lesRepublicains Pour DÉGAGER MACRON  : #ZemmourMobilisationGenerale #ZemmourPresident2022 #Zemmour2emeTour',
 "Petit travail artistique du jour. J'étais inspirée 😁\n\nLES ARTISTES AUSSI PEUVENT ÊTRE POUR ZEMMOUR 💖 GZ 💖\n\n#JeVoteZemmour #JeVotePourZemmour #ZemmourMobilisationGenerale #ZemmourPresident2022 #LesFemmesAvecZemmour #Zemmour2022 #Zemmour2emeTour #ZemmourLeVoteVital #ZemmourEric https://t.co/rKGiZl2RSK",
 '@kalyne2403 Aucun souci #ZemmourPresident2022 #Zemmour2022 #Zemmour2emeTour',
 

In [ ]:
# # # # # # # # #  BELOW IS NOT GOOD WORDS 

In [207]:
df_prop.loc[df_prop['target_larger'] == False].sort_values('llik',ascending = False).head(20)

,target_freq,target_num_docs,target_num_docs_prop,not_target_freq,not_target_num_docs,not_target_num_docs_prop,ref_freq,ref_num_docs,ref_num_docs_prop,target_larger,llik
zemmourpresid,14301,14216,0.039012,242220,236202,0.263319,345,345,0.310252,False,-706871.414781
zemmourprsid,99,99,0.000272,20007,19771,0.022041,22,22,0.019784,False,-752010.608028
melenchonvagagn,2090,2089,0.005733,31127,30885,0.034431,52,52,0.046763,False,-752864.046717
zemmour,32496,29873,0.081978,147914,132192,0.147368,231,190,0.170863,False,-752971.150628
forc zemmourpresid,2,2,0.000005,9505,9505,0.010596,4,4,0.003597,False,-755074.951515
montrez votr,3,3,0.000008,9496,9496,0.010586,4,4,0.003597,False,-755084.870952
votr forc,2,2,0.000005,9471,9471,0.010558,4,4,0.003597,False,-755086.636738
montrez,72,72,0.000198,9648,9646,0.010753,4,4,0.003597,False,-755344.292778
zemmour zemmourpresid,1780,1773,0.004866,19527,19467,0.021702,31,31,0.027878,False,-755535.119801
lafrancedanslesyeux,431,431,0.001183,11771,11758,0.013108,0,0,0.000000,False,-755612.663393


In [176]:

def see_in_context(term):
    '''Gets the tweets orginial text with the term - also supports regex - seachers the full tect
    only returns tweets not in the orginial reference list'''
    if list(vectorizer.get_feature_names_out()):
        subset = df.loc[(df['text_original'].str.contains(pat = term,regex = True, case = False)) & (df['reference_set'] == False),'text_original']
        if len(subset) != 0:
            return subset
        else:
            print("This is not a term in the corpus")

    else:
        print("This is not a term in the corpus")



In [95]:
see_in_context("montrez votr").iloc[1]

"@Le_Figaro ⌛ Z-118j #Zemmour\n\n📰: La tiers-mondisation 🗺️ de notre pays et de notre peuple l'appauvrît autant qu'elle le disloque, le ruine autant qu'elle le tourmente ✝️⚜️🇫🇷\n\n🚀 RT = une voix pour le 🇿 !\n💯 Like = une signature #Z0ZZ !\n\n🇫🇷 Montrez votre force #ZemmourPresident ‼️"

In [136]:
df_elites = pd.read_excel('Elite actors list and keywords_2023.10.06.15.30.xlsx', sheet_name='all_mps')

In [137]:
df_elites.head()

,Twitter,Position,Affilation
0,https://twitter.com/PapNdiaye,Government,LFM
1,https://twitter.com/sretailleau,Government,LFM
2,https://twitter.com/RimaAbdulMalak,Government,LFM
3,https://twitter.com/E_DupondM,Government,LFM
4,https://twitter.com/MinColonna,Government,LFM


In [138]:
df_elites['handles'] = df_elites['Twitter'].str.extract(r'https:\/\/twitter\.com\/(\w+)')

In [139]:
df_elites.head()

,Twitter,Position,Affilation,handles
0,https://twitter.com/PapNdiaye,Government,LFM,PapNdiaye
1,https://twitter.com/sretailleau,Government,LFM,sretailleau
2,https://twitter.com/RimaAbdulMalak,Government,LFM,RimaAbdulMalak
3,https://twitter.com/E_DupondM,Government,LFM,E_DupondM
4,https://twitter.com/MinColonna,Government,LFM,MinColonna


In [140]:
len(df_elites)

552

In [141]:
df_elites.columns

Index(['Twitter', 'Position', 'Affilation', 'handles'], dtype='object')

In [142]:
df.head()

,source,target,weight,text,month,text_original,includes,excludes,reference_set,text_preprossed,text_pre_stem
0,Mehetia1,None,1,etudiants: la #france devenue un pays du tier...,December,@demontvalon1 @EmmanuelMacron Etudiants : La #...,True,False,1,etudiants: la #france devenue un pays du tier...,etudiants: la #franc devenu un pay du tiers-mo...
1,LORZMichael_Off,None,1,une honte. hidalgo devrait tre vir au plus vit...,December,Une honte. Hidalgo devrait être viré au plus v...,False,False,0,une honte. hidalgo devrait tre vir au plus vit...,une honte. hidalgo devrait tre vir au plu vite...
2,Thomas_France75,None,1,ne plus fermer les yeux #presidentielles2022,December,Ne plus fermer les yeux #presidentielles2022🇫🇷👇🏼,True,False,1,ne plus fermer les yeux #s2022,ne plu fermer le yeux #s
3,Fmercier2017,None,1,"le nuclaire est certes productif, mais trop pr...",December,"Le nucléaire est certes productif, mais trop p...",False,False,0,"le nuclaire est certes productif, mais trop pr...","le nuclair est cert productif, mai trop problm..."
4,BerbreReineKah1,None,1,que c'est beau de rverelle est belle la start-...,December,Que c'est beau de rêver😢\nElle est belle la st...,False,False,0,que c'est beau de rverelle est belle la start-...,que c'est beau de rverel est bell la start-up ...


In [157]:
import pandas as pd

# Assuming df and df_elites are your DataFrames

# Initialize an empty list to store the results
elite_actors_tweet_nb = []

# Loop through unique handles in df_elites
for handle in df_elites['handles'].unique():
    # Count the number of rows where 'source' matches the current handle
    matching_rows = df[df['source'] == handle].shape[0]
    
    # Check if there are matching rows in df_elites for the current handle
    if len(df_elites[df_elites['handles'] == handle]) > 0:
        # Get the affiliation and position for the current handle
        affiliation = df_elites.loc[df_elites['handles'] == handle, 'Affilation'].values[0]
        position = df_elites.loc[df_elites['handles'] == handle, 'Position'].values[0]
    else:
        affiliation = None  # Set affiliation as None if there are no matching rows
        position = None  # Set position as None if there are no matching rows
    
    # Append handle, sum, affiliation, and position to the results list
    elite_actors_tweet_nb.append({'handle': handle, 'sum': matching_rows, 'Affilation': affiliation, 'Position': position})

# Create a new DataFrame from the results list
elite_actors_tweet_nb_df = pd.DataFrame(elite_actors_tweet_nb)

# Print the new DataFrame
print(elite_actors_tweet_nb_df)


              handle  sum Affilation                Position
0          PapNdiaye    0        LFM              Government
1        sretailleau    0        LFM              Government
2     RimaAbdulMalak    0        LFM              Government
3          E_DupondM    3        LFM              Government
4         MinColonna    0        LFM              Government
..               ...  ...        ...                     ...
498             EELV  218       EELV        Official Account
499           yjadot  837       EELV            Party Leader
500  lesrepublicains    0         LR  Official Party Account
501        vpecresse  519         LR            Party Leader
502              NaN    0       None                    None

[503 rows x 4 columns]


In [158]:
elite_actors_tweet_nb_df.sort_values(by='sum', ascending=False)

,handle,sum,Affilation,Position
499,yjadot,837,EELV,Party Leader
295,MLP_officiel,669,RN,Party Leader
501,vpecresse,519,LR,Party Leader
381,JulienOdoul,421,RN,MP
357,ljacobelli,246,RN,MP
...,...,...,...,...
283,MickaelNogal,0,LFM,MP
286,LoicDombreval,0,LFM,MP
101,MarieAngeMagne,0,LFM,MP
108,Fdumas2017,0,LFM,MP


In [159]:
len(elite_actors_tweet_nb_df[elite_actors_tweet_nb_df['sum'] == 0])

134

In [160]:
elite_actors_tweet_nb_df.to_excel('elite_actors_tweet_nb_3.xlsx')